In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

work_dir = r'D:\Documents\course\junior\BigDataAna\12-20大作业'
dataset_dir = work_dir+r'\datasets'+'\\'
# print(dataset_dir)

# 获取数据集
raw_data = pd.read_csv(dataset_dir+'movies.csv')
raw_data.info()
print(raw_data.head())
index2Id = {k:v for k, v in enumerate(raw_data['movieId'])}
Id2index = {v:k for k, v in index2Id.items()}

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [16]:
minihash = True
genres = raw_data['genres'].values

In [17]:
train_data = pd.read_csv(dataset_dir+'train_set.csv')
train_data.info()
train_data.drop('timestamp', axis=1, inplace=True)
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99904 entries, 0 to 99903
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     99904 non-null  int64  
 1   movieId    99904 non-null  int64  
 2   rating     99904 non-null  float64
 3   timestamp  99904 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.0 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99904 entries, 0 to 99903
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   99904 non-null  int64  
 1   movieId  99904 non-null  int64  
 2   rating   99904 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [47]:
proj_func_num = 20
mat = train_data.pivot_table(index=['userId'], columns=['movieId'], values=['rating'])
mat.fillna(0,inplace=True)
util_mat = np.array(mat.values)
movie_id2index = {mat.columns[i][1]:i for i in range(util_mat.shape[1])}
index2movie_id = {v:k for k,v in movie_id2index.items()}
user_num = util_mat.shape[0]
movie_num = util_mat.shape[1]

In [48]:

# 获取tfidf特征矩阵
vectorizer = TfidfVectorizer()
tfidf_mat = vectorizer.fit_transform(genres).toarray()
tfidf_mat

array([[0.        , 0.41043288, 0.5315272 , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.510466  , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [49]:
from time import time
from scipy.spatial.distance import pdist,squareform
start_time = time()
if minihash:

    # 构建哈希签名矩阵
    mini_mat = np.array(tfidf_mat)
    mini_mat[mini_mat>0] = 1
    mini_mat[mini_mat<=0] = 0
    total_movie_num = mini_mat.shape[0]
    feature_num = mini_mat.shape[1]
    sig_mat = np.zeros((proj_func_num, total_movie_num))
    for i in range(proj_func_num):
        print("pai%d/%d" % (i+1, proj_func_num))
        pai = [j for j in range(feature_num)]
        np.random.shuffle(pai)
        pai_index = np.argsort(pai)
        for j in range(total_movie_num):
            for index in pai_index:
                if mini_mat[j, pai[index]] == 1:
                    sig_mat[i, j] = pai[index]
                    break
    # 计算jiccard相似度
    print('calc sim mat')
    sig_mat = sig_mat.T # 用行来算比较快
    sim_mat = 1 - squareform(pdist(sig_mat, 'jaccard'))
    print('sim mat cacl finish')
    sim_mat[np.isnan(sim_mat)] = 0
    sim_mat

else:
    #根据特征矩阵计算相似度矩阵
    sim_mat = cosine_similarity(tfidf_mat)
    sim_mat[sim_mat<0] = 0

end_time = time()
print("use time:",end_time-start_time)

pai1/20
pai2/20
pai3/20
pai4/20
pai5/20
pai6/20
pai7/20
pai8/20
pai9/20
pai10/20
pai11/20
pai12/20
pai13/20
pai14/20
pai15/20
pai16/20
pai17/20
pai18/20
pai19/20
pai20/20
calc sim mat
sim mat cacl finish
use time: 6.487042188644409


In [38]:
K = 5
n = 5
def recommender(mode, userId, util_mat, sim_mat, movie_id):
    userId = userId-1  # userID 从1开始，矩阵从0开始
    user_num = util_mat.shape[0]
    movie_num = util_mat.shape[1]
    if mode == 1: # 1为top k推荐 0为预测评分 2为预测单个movie的评分
        user_rate = np.zeros(util_mat.shape[1])
    else:
        user_rate = np.copy(util_mat[userId])

    rate_movie_index = []
    unrate_movie_index = []
    for cur_movie in range(movie_num):
        if util_mat[userId, cur_movie] == 0 and mode != 2:
            unrate_movie_index.append(cur_movie)
        else:
            rate_movie_index.append(cur_movie)

    if mode == 2:
        unrate_movie_index.append(movie_id2index[movie_id])

    for cur_movie in unrate_movie_index:
        total = 0
        pred = 0
        for another_movie in rate_movie_index:
            another_movie_rate = util_mat[userId, another_movie]
            if another_movie_rate != 0:
                cur_movie_in_sim_mat = Id2index[index2movie_id[cur_movie]]
                another_movie_in_sim_mat = Id2index[index2movie_id[another_movie]]
                pred += sim_mat[cur_movie_in_sim_mat, another_movie_in_sim_mat]*another_movie_rate
                total += sim_mat[cur_movie_in_sim_mat, another_movie_in_sim_mat]
                # if cur_movie == 3303:
                # print('pred',pred)
                # print('total',total)
                # print('cur_movie:', cur_movie)
                # print('another user rate:', another_movie_rate)
                # print('sim:', sim_mat[cur_movie_in_sim_mat, another_movie_in_sim_mat])
        # if cur_movie == 3303:
        # print('pred',pred)
        # print('total',total)
        # print('rate', pred/total)
        # print('cur_movie:', cur_movie)
        user_rate[cur_movie] = pred/total

    if mode == 0 or mode == 2:
        # user_rate[np.isnan(user_rate)] = 0
        return user_rate
    else:
        user_rate[np.isnan(user_rate)] = 0
        rec_index = np.argsort(user_rate)[-K:]
        print(rec_index)
        print(user_rate[rec_index])
        print(user_rate)
        movie_id_list = list(mat.columns)
        rec = [movie_id_list[i][1] for i in rec_index]
        return rec




recommender(1, 600,util_mat, sim_mat,0)

D:\Programs\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


[ 308 7710  809  818 7427]
[5. 5. 5. 5. 5.]
[4.3125 4.3125 2.5    ... 0.     4.3125 0.    ]


[343, 84312, 1005, 1014, 74282]

In [39]:
# 读取测试集
test = pd.read_csv(dataset_dir+'test_set.csv')
test.drop('timestamp', axis=1, inplace=True)
users, movies, ratings = test['userId'], test['movieId'], test['rating']
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   100 non-null    int64  
 1   movieId  100 non-null    int64  
 2   rating   100 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 2.5 KB


In [40]:
# 开始预测
preds = []
pred_cache = {}
for i in range(len(test)):
    print('%d/%d...' % (i+1, len(test)))
    # if users[i] not in pred_cache.keys():
    rates = recommender(2, users[i], util_mat, sim_mat, movie_id=movies[i])
    preds.append(rates[movie_id2index[movies[i]]])
    # pred_cache[users[i]] = rates
    # else:
    #     rates = pred_cache[users[i]]
    #     preds.append(rates[movie_id2index[movies[i]]])

SSE = np.sum(np.square(preds - ratings))

SSE

1/100...
2/100...
3/100...
4/100...
5/100...
6/100...
7/100...
8/100...
9/100...
10/100...
11/100...
12/100...
13/100...
14/100...
15/100...
16/100...
17/100...
18/100...
19/100...
20/100...
21/100...
22/100...
23/100...
24/100...
25/100...
26/100...
27/100...
28/100...
29/100...
30/100...
31/100...
32/100...
33/100...
34/100...
35/100...
36/100...
37/100...
38/100...
39/100...
40/100...
41/100...
42/100...
43/100...
44/100...
45/100...
46/100...
47/100...
48/100...
49/100...
50/100...
51/100...
52/100...
53/100...
54/100...
55/100...
56/100...
57/100...
58/100...
59/100...
60/100...
61/100...
62/100...
63/100...
64/100...
65/100...
66/100...
67/100...
68/100...
69/100...
70/100...
71/100...
72/100...
73/100...
74/100...
75/100...
76/100...
77/100...
78/100...
79/100...
80/100...
81/100...
82/100...
83/100...
84/100...
85/100...
86/100...
87/100...
88/100...
89/100...
90/100...
91/100...
92/100...
93/100...
94/100...
95/100...
96/100...
97/100...
98/100...
99/100...
100/100...


D:\Programs\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:44: RuntimeWarning: invalid value encountered in double_scalars


52.71847575457196